Задание 8.1 Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [70]:
import pandas as pd
diabetes = pd.read_csv('data/diabetes_data.csv')
#display(diabetes.head())
#display(diabetes.shape)
#display(diabetes.info())
dupl_columns_diabetes = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns_diabetes)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns_diabetes)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')
#display(diabetes_dedupped.head())
#display(diabetes_dedupped.info())
#display(diabetes_dedupped.shape)

Число найденных дубликатов: 10
Результирующее число записей: 768


Задание 8.2 Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [60]:
low_information_cols = [] #список неинформативных признаков

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
information_diabetes = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')
#display(information_diabetes.head())
#display(information_diabetes.info())
display(information_diabetes.shape)

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


(768, 9)

Задание 8.3 Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [71]:
information_diabetes_NaN = information_diabetes.copy()

import numpy as np
def IsNullValue(val):
    if val==0:
        return np.nan
    else:
        return val
information_diabetes_NaN['Glucose'] = information_diabetes_NaN['Glucose'].apply(IsNullValue)
information_diabetes_NaN['BloodPressure'] = information_diabetes_NaN['BloodPressure'].apply(IsNullValue) 
information_diabetes_NaN['SkinThickness'] = information_diabetes_NaN['SkinThickness'].apply(IsNullValue)
information_diabetes_NaN['Insulin'] = information_diabetes_NaN['Insulin'].apply(IsNullValue)    
information_diabetes_NaN['BMI'] = information_diabetes_NaN['BMI'].apply(IsNullValue)  
#print(information_diabetes_NaN.head())
#print(information_diabetes_NaN.info())
display(information_diabetes_NaN.shape)


(768, 9)

Задание 8.4
Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [80]:
information_diabetes_drop = information_diabetes_NaN.copy()
thresh_diabetes = information_diabetes_drop.shape[0]*0.7#задаем минимальный порог: вычисляем 70% от числа строк
information_diabetes_drop = information_diabetes_drop.dropna(thresh=thresh_diabetes, axis=1)#удаляем столбцы, в которых более 30% (100-70) пропусков
#information_diabetes_drop = information_diabetes_drop.dropna(how='any', axis=0)
#print(information_diabetes.isnull().mean())
print(f'Результирующее число признаков: {information_diabetes_drop.shape[1]}')
#display(information_diabetes_drop.info())
display(information_diabetes_drop.shape)
display(information_diabetes_drop)

Результирующее число признаков: 8


(768, 8)

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


Задание 8.5
Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [90]:
information_diabetes_drop_no_more_two = information_diabetes_drop.copy()

m = information_diabetes_drop_no_more_two.shape[1]
information_diabetes_drop_no_more_two = information_diabetes_drop_no_more_two.dropna(thresh=m-2,axis=0)
print(information_diabetes_drop_no_more_two.shape)

print(f'Результирующее число признаков: {information_diabetes_drop_no_more_two.shape[0]}')
#display(information_diabetes_drop_no_more_two.head())
display(information_diabetes_drop_no_more_two.info())

(761, 8)
Результирующее число признаков: 761
<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


None

Задание 8.6 В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [97]:
information_diabetes_fill_median=information_diabetes_drop_no_more_two.copy()#создаём копию исходной таблицы

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'Glucose': information_diabetes_fill_median['Glucose'].median(),
    'BloodPressure': information_diabetes_fill_median['BloodPressure'].median(),
    'SkinThickness': information_diabetes_fill_median['SkinThickness'].median(),
    'BMI': information_diabetes_fill_median['BMI'].median()
}

information_diabetes_fill_median = information_diabetes_fill_median.fillna(values)
print(round(information_diabetes_fill_median.mean(),1))
#print(information_diabetes_fill_median.head())
#print(information_diabetes_fill_median.shape)

Pregnancies                   3.8
Glucose                     121.9
BloodPressure                72.4
SkinThickness                29.1
BMI                          32.5
DiabetesPedigreeFunction      0.5
Age                          33.3
Outcome                       0.4
dtype: float64


Задание 8.7 Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [98]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr(information_diabetes_fill_median, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Задание 8.8 Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [100]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score(information_diabetes_fill_median, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


Задание 8.9  На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

In [102]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr(information_diabetes_fill_median, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score(information_diabetes_fill_median, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761
